# VBA Benchmark Model

In [3]:
import numpy
import xlrd
from openpyxl import load_workbook
from vbaFunc import jumpoff, xTFR, clear_Input
from vbaFunc import readCNAPC, readSYI, readSATP, readERP, readTFR, readASFR, readLEMS, readMASMR
from vbaFunc import inputASFR, inputbirth, inputASDR, inputdeath, inputMigration, writeAccount, writeSAI
from vbaFunc import NetMigAdjustment2, readNatP, readBDN, readIniPop, readIniTPop, writeNoteCL, writeProj
from vbaFunc import writeTarget

# Read in Excel Workbook (both Readable and Writable)
wt_loc = 'vba.xlsx'
rd_loc = 'original.xlsm'
wb_rd = xlrd.open_workbook(rd_loc)
wb_wt = load_workbook(wt_loc)

## xTRF

#### Generating Fertility Data

In [4]:
# Read in Sheets
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
wb_wt_fertility = wb_wt["Fertility"]

# Define variables
numareas = int(sheet_label.cell_value(2256, 0))
numages = 18
sextypes = 2
set_year_female = 7
set_year_male = 5

# Create empty array for storing jumpoffERP and xTFR results
jumpoffERP = numpy.zeros((numareas, sextypes, numages))
jumpoffERP[:] = numpy.nan
result_xTFR = numpy.zeros((numareas))
result_xTFR[:] = numpy.nan

# Generate jumpoffERP for further calculation
jumpoffERP = jumpoff(numareas, numages, jumpoffERP, sheet_agesex, set_year_female, set_year_male)

# Generate xTFR vector for storing
result_xTFR = xTFR(jumpoffERP, result_xTFR, numareas)

# Store into Sheet 'Fertility' for further usage
row = 7
column = 4
for i in range(len(result_xTFR)):
    wb_wt_fertility.cell(row, column).value = result_xTFR[i]
    row += 1
wb_wt.save(wt_loc)

## PrepareData

#### Clear Previous Input Data

In [5]:
# Before running the PrepareData Function, clear the input data if any
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_SmallAreaInputs)
clear_Input(wb_wt_Accounts)
wb_wt.save(wt_loc)

#### Load in Required Data for Generating Input Data

In [6]:
# Read in readable required data for generating input data for further usage
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_SmallAreaTotals = wb_rd.sheet_by_name('SmallAreaTotals')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
sheet_mortality = wb_rd.sheet_by_name('Mortality')
sheet_migration = wb_rd.sheet_by_name('Migration')

# Temeorary read of new generated fertility
temp_wb_rd = xlrd.open_workbook(wt_loc)
sheet_fertility = temp_wb_rd.sheet_by_name('Fertility')

#### Start Generating Input Data

##### Define Variables

In [7]:
numages = 18                                    # Number of age groups in ERPs
lastage = 17                                    # Number of age groups in projections output
nLxMS = 21                                      # nLx mortality surface groups
age_ASFR = 7                                    # Age groups amount in ASFRs Model
numareas = int(sheet_label.cell_value(2256, 0)) # Number of small areas
final_year = 2035                               # Projection final year (editable)
final = int((final_year - 2020) / 5)            # Final projection year number
SRB = sheet_fertility.cell_value(2270, 1)       # Assumed Sex Ratio at Birth (Male : Female)
modelTFR  = 0                                   # Model TFR Parameter
for i in range(2259, 2266): modelTFR += sheet_fertility.cell_value(i, 1)
modelTFR *= 5

##### Read in Data for Generating Input Data

In [8]:
## All two dimensional array with 0 and 1 as sex index, we treat 0 as female and 1 as male
# Read in AreaCode, AreaName, AgeGroup, Period-Cohort Labels
Areacode, Areaname, agelabel, pclabel = readCNAPC(numareas, numages, lastage, sheet_label)

# Read in Sex, Year, Interval Labels
sexlabel, yearlabel, intervallabel = readSYI(final, sheet_label)

# Read in Estimated & Projected Small Area Total Population (Aggregated)
TotPop = readSATP(2, sheet_SmallAreaTotals, numareas)

# Read in small area ERP (Estimated Resident Population) by age & sex (before and at jumpoff year)
ERP = readERP(2, numareas, numages, sheet_agesex, False)

# Area-specific TFRs
TFR = readTFR(final, numareas, sheet_fertility)

# Read in data for ASFRs and Preliminary ASFRs (Area specific Fertility Rate)
modelASFR, prelimASFR = readASFR(numareas, sheet_fertility, age_ASFR)

# Read in data for Life Expectancy and Mortality Surface
eO, MS_nLx, MS_TO = readLEMS(final, numareas, lastage, nLxMS, sheet_mortality)

# Read in data of migration data and for ASMRs (Area specific Mortality Rate)
totmig, modelASMR = readMASMR(numareas, lastage, sheet_migration)

##### Some Checks

In [9]:
# TFRs > 0
for i in range(final + 1):
    for a in range(numareas):
        if (TFR[i, a] < 0.1):
            print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a])
            break
    break

# eOs > 0
for i in range(final + 1):
    for a in range(numareas):
        for q in range(2):
            if (eO[i, a, q] < 0.1):
                print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a], " & Sex = ", sexlabel[q])
                break
        break
    break

# Check Migration Turnover > 0
for i in range(numareas):
    if (totmig[i] < 0.1):
        print("Error: Area =", Areaname[i])
    break
    
# Check Small Area Population Total > 0
for i in range(2):
    for a in range(numareas):
        if (TotPop[i, a] < 0.1):
            print("Error Year =", yearlabel[i], " & Area = ", Areaname[a])
        break
    break

##### Calculate Input Data

In [10]:
# Create input data for ASFR (Area specific Fertility Rate)
ASFR = inputASFR(final, numareas, age_ASFR, sheet_fertility, prelimASFR, modelASFR, TFR, modelTFR)

# Estimate base period birth by sex for the population 'Accounts'
tempbirthssex, temptotbirths = inputbirth(numareas, age_ASFR, ASFR, ERP, SRB)

# Calculate base period projected age-specific death rate (used in estimating death account)
ASDR = inputASDR(final, numareas, lastage, nLxMS, eO, MS_TO, MS_nLx)

# Calculate estimated death for age-sex cohorts and total population accounts
tempdeaths, temptotdeaths = inputdeath(numareas, lastage, ASDR, ERP)

# Calculate migration input data
ASOMR, inward, outward, prelimmig, scaledmig, prelimbaseinward, prelimbaseoutward, cohortnetmig, adjnetmig, scalingfactor2a, scalingfactor2b = \
inputMigration(final, numareas, lastage, modelASMR, ERP, totmig, TotPop, temptotdeaths, temptotbirths, tempdeaths, tempbirthssex, ASDR, Areaname, sexlabel, pclabel, intervallabel)

##### Write Input Data

In [11]:
# Write Input Data to Accounts Sheet
wb_wt_Accounts = writeAccount(wb_wt_Accounts, numareas, Areaname, lastage, sexlabel, pclabel, tempbirthssex, inward, outward, prelimmig, scaledmig, prelimbaseinward, prelimbaseoutward, cohortnetmig, adjnetmig, scalingfactor2a, scalingfactor2b, ERP, tempdeaths)

# Write Input Data to SmallAreaInputs Sheet
wb_wt_SmallAreaInputs = writeSAI(wb_wt_SmallAreaInputs, intervallabel, numareas, lastage, final, Areacode, Areaname, pclabel, ASDR, ASOMR, inward, agelabel, ASFR)

# Save the written workbook
wb_wt.save(wt_loc)
print("Input data is prepared")

Input data is prepared


## SyntheticProjectonModel

#### Clear Previous Output Data

In [12]:
# Before running the SyntheticProjectionModel Function, clear the output data if any
wb_wt_AgeSexForecasts = wb_wt["AgeSexForecasts"]
wb_wt_Components = wb_wt["Components"]
wb_wt_Target = wb_wt["Target"]
wb_wt_CheckMig = wb_wt["CheckMig"]
wb_wt_CheckDeaths = wb_wt["CheckDeaths"]
wb_wt_Check_OK = wb_wt["Check_OK"]
wb_wt_Log = wb_wt["Log"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_AgeSexForecasts)
clear_Input(wb_wt_Components)
clear_Input(wb_wt_CheckMig)
clear_Input(wb_wt_CheckDeaths)
clear_Input(wb_wt_Check_OK)
clear_Input(wb_wt_Log)
wb_wt.save(wt_loc)

#### Load in Required Data for Generating Projection Data

In [13]:
# Read in readable required data for generating input data for further usage
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_SmallAreaTotals = wb_rd.sheet_by_name('SmallAreaTotals')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
sheet_mortality = wb_rd.sheet_by_name('Mortality')
sheet_migration = wb_rd.sheet_by_name('Migration')
sheet_NationalProjection = wb_rd.sheet_by_name('NationalProjection')

# Temeorary read of new generated fertility
temp_wb_rd = xlrd.open_workbook(wt_loc)
sheet_fertility = temp_wb_rd.sheet_by_name('Fertility')
sheet_accounts = temp_wb_rd.sheet_by_name('Accounts')
sheet_SmallAreaInputs = temp_wb_rd.sheet_by_name('SmallAreaInputs')

#### Start Generating Projection Data

##### Define Variables

In [14]:
smallnumber = 0.0001                            # Smoothed Number
maxqiter = int(sheet_label.cell_value(10, 10))  # Max number of population-at-risk loop iterations
maxziter = int(sheet_label.cell_value(13, 10))  # Max number of migration adjustment iterations
numages = 18                                    # Number of age groups in ERPs
lastage = 17                                    # Number of age groups in projections output
nLxMS = 21                                      # nLx mortality surface groups
age_ASFR = 7                                    # Age groups amount in ASFRs Model
numareas = int(sheet_label.cell_value(2256, 0)) # Number of small areas
final_year = 2035                               # Projection final year (editable)
final = int((final_year - 2020) / 5)            # Final projection year number
SRB = sheet_fertility.cell_value(2270, 1)       # Assumed Sex Ratio at Birth (Male : Female)
modelTFR  = 0                                   # Model TFR Parameter
for i in range(2259, 2266): modelTFR += sheet_fertility.cell_value(i, 1)
modelTFR *= 5

##### Load in required data for generating projection output

In [15]:
## All two dimensional array with 0 and 1 as sex index, we treat 0 as female and 1 as male
# Read in AreaCode, AreaName, AgeGroup, Period-Cohort Labels
Areacode, Areaname, agelabel, pclabel = readCNAPC(numareas, numages, lastage, sheet_label)

# Read in Sex, Year, Interval Labels
sexlabel, yearlabel, intervallabel = readSYI(final, sheet_label)

# Read in Estimated & Projected Small Area Total Population (Aggregated)
TotPop = readSATP(4, sheet_SmallAreaTotals, numareas)

# Read in small area ERP (Estimated Resident Population) by age & sex (before and at jumpoff year)
ERP = readERP(1, numareas, numages, sheet_agesex, True)

# Read in National population estimate & projections by age & sex
NatP = readNatP(final, numages, sheet_NationalProjection)

# Read in ASDR, ASOMR, Inward Migration, ASFR data for projection
ASDR = ASDR[1:, 0:, 0:, 0:]
ASOMR = ASOMR[1:, 0:, 0:, 0:]
inward = inward[1:, 0:, 0:, 0:]
ASFR = ASFR[1:, 0:, 0:]

# Read in National projected births, deaths, net migration data
NatB, NatDpc, NatNpc = readBDN(final, lastage, sheet_NationalProjection)

##### Some Checks & Headings Writing

In [16]:
# pop total > 0
for y in range(final):
    for i in range(numareas):
        if (TotPop[y + 1, i] < 0.1):
            print("Error: Year = ", yearlabel[y + 2], " & Area = ", Areaname[i])
            break
    break

# Check small area population totals match national population total
for y in range(final + 1):
    checktot1 = 0
    for i in range(numareas):
        checktot1 += TotPop[y, i]
    checktot2 = 0
    for s in range(2):
        for a in range(lastage + 1):
            checktot2 += NatP[y, s, a]
    
    if (abs(checktot1 - checktot2) > smallnumber):
        print("Error: Year = ", yearlabel[y + 1], "Sum of pop total = ", checktot1, ", Sum of national projections = ", checktot2)
        break
    break

# Write headings and row names into the selected sheet(s)
wb_wt_CheckMig, wb_wt_CheckDeaths, wb_wt_Log = writeNoteCL(numareas, lastage, wb_wt_CheckMig, wb_wt_CheckDeaths, wb_wt_Log, intervallabel, sexlabel, pclabel, Areaname, final)
wb_wt.save(wt_loc)

##### Start Projection

In [17]:
# Set initial value of jump-off year population in age-sex cohort
Population = readIniPop(final, numareas, numages, ERP)

# Record total jump-off population into dataset for further projection usage
totPopulation = readIniTPop(final, numareas, numages, Population)

# Main Projection Section Starts
# Start the Projection Interval Loop (record final = 3 projection years including 2025, 2030, 2035)
Pop0 = numpy.zeros((final, numareas, 2, lastage + 1))
Pop1 = numpy.zeros((final, numareas, 2, lastage + 1))
Btot = numpy.zeros((final, numareas))                      # Total Birth in sex cohort
Dpc = numpy.zeros((final, numareas, 2, lastage + 1))       # Death in age-sex cohort
Dtot = numpy.zeros((final, numareas))                      # Total Death in sex cohort
IMpc = numpy.zeros((final, numareas, 2, lastage + 1))      # Total Inward Migration in sex cohort
OMpc = numpy.zeros((final, numareas, 2, lastage + 1))      # Total Outward Migration in sex cohort
totN = numpy.zeros((final, numareas))
previousPop1 = numpy.zeros((numareas, 2, lastage + 1))
totINMIG = numpy.zeros((final, numareas))
totOUTMIG = numpy.zeros((final, numareas))

for y in range(1, final + 1):

   # Record for each year of projection, the initial population is the previous year's projected population
   # This function does not record the projected pop of 0-4 group
   for i in range(numareas):
      for s in range(2):
         for pc in range(1, lastage):
               Pop0[y - 1, i, s, pc] = Population[y - 1, i, s, pc - 1]
         
         # Record 85+ group population
         Pop0[y - 1, i, s, lastage] = Population[y - 1, i, s, lastage - 1] + Population[y - 1, i, s, lastage]
   
   # Start large iteraction
   for q in range(1, maxqiter + 1):

      # Record the number of iteration required
      wb_wt_Log.cell(3 + y, 1).value = intervallabel[y]
      wb_wt_Log.cell(3 + y, 2).value = q
      
      # For the first iteration, the end-of-interval population is set (avoid a negative projected population)
      if (q == 1):
         for i in range(numareas):
            for s in range(2):
               for pc in range(lastage):
                     Pop1[y - 1, i, s, pc + 1] = 0 # (+++ Projection Data Pop1 is written +++)
                        
      #-------------------------------------------------------------------------------------------------------------------------
      # Birth Projection
      # Preliminary birth by age of mother
      Bage = numpy.zeros((numareas, 7))
      Btottmp = numpy.zeros((numareas))
      for i in range(numareas):
         Btottmp[i] = 0
         for pc in range(7):
            Bage[i, pc] = ASFR[y - 1, i, pc] * 2.5 * (Pop0[y - 1, i, 0, pc + 4] + Pop1[y - 1, i, 0, pc + 3])
            Btottmp[i] += Bage[i, pc]
         
      # Preliminary birth by sex
      Bsextmp = numpy.zeros((numareas, 2))
      Bsextmptot =  numpy.zeros((2))
      for i in range(numareas):
         # Female
         Bsextmp[i, 0] = Btottmp[i] * (100 / (100 + SRB))
         Bsextmptot[0] += Bsextmp[i, 0]
         # Males
         Bsextmp[i, 1] = Btottmp[i] * (SRB / (100 + SRB))
         Bsextmptot[1] += Bsextmp[i, 1]

      # Constrain to national births by sex (+++ Projection total birth -- Btot is Written +++)
      Bsex = numpy.zeros((numareas, 2))
      for i in range(numareas):
         Btot[y - 1, i] = 0
         for s in range(2):
            Bsex[i, s] = Bsextmp[i, s] * NatB[y - 1, s] / Bsextmptot[s]
            Btot[y - 1, i] += Bsex[i, s]
            
      # Allocate births to the initial infant population (+++ Projection Pop0 is written for filling age-group 0-4 +++)
      for i in range(numareas):
         for s in range(2):
            Pop0[y - 1, i, s, 0] = Bsex[i, s]
      
      # For the first iteration, the end-of-interval group 0-4 population is set (+++ Pop1 is written fo filling group 0-4 +++)
      if (q == 1):
         for i in range(numareas):
            for s in range(2):
               Pop1[y - 1, i, s, 0] = Bsex[i, s]
      
      #-------------------------------------------------------------------------------------------------------------------------
      # Death Projection
      # Preliminary deaths in age-sex groups
      Dtmp = numpy.zeros((numareas, 2, lastage + 1))
      for i in range(numareas):
         for s in range(2):
            # Record age group without 0-4
            for pc in range(1, lastage + 1):
               Dtmp[i, s, pc] = ASDR[y - 1, i, s, pc] * 2.5 * (Pop0[y - 1, i, s, pc] + Pop1[y - 1, i, s, pc])
            # Record age group 0-4    
            Dtmp[i, s, 0] = ASDR[y - 1, i, s, 0] * 2.5 * Pop1[y - 1, i, s, 0]
      
      # Preliminary total deaths (sum age)
      Dtmptot = numpy.zeros((2, lastage + 1))
      for i in range(numareas):
         for s in range(2):
            for pc in range(lastage + 1):
               Dtmptot[s, pc] += Dtmp[i, s, pc]
      
      # Constrain to national deaths (+++ Dpc, Dtot are written as death projection data +++)
      for i in range(numareas):
         Dtot[y - 1 , i] = 0
         for s in range(2):
            for pc in range(lastage + 1):
               Dpc[y - 1, i, s, pc] = Dtmp[i, s, pc] * NatDpc[y - 1, s, pc] / Dtmptot[s, pc]
               Dtot[y - 1, i] += Dpc[y - 1, i, s, pc]

      #-------------------------------------------------------------------------------------------------------------------------
      # Migration Projection
      # Preliminary Migration (in & out)
      prelimIMpc = numpy.zeros((numareas, 2, lastage + 1))
      prelimOMpc = numpy.zeros((numareas, 2, lastage + 1))
      for i in range(numareas):
         for s in range(2):
            # Record age group without 0-4
            for pc in range(1, lastage + 1):
               prelimIMpc[i, s, pc] = inward[y - 1, i, s, pc]
               prelimOMpc[i, s, pc] = ASOMR[y - 1, i, s, pc] * 2.5 * (Pop0[y - 1, i, s, pc] + Pop1[y - 1, i, s, pc])
            # Record age group 0-4
            prelimIMpc[i, s, 0] = inward[y - 1, i, s, 0]
            prelimOMpc[i, s, 0] = ASOMR[y - 1, i, s, 0] * 2.5 * Pop1[y - 1, i, s, 0]
      
      # Calculate residual net migration required to produce total projected populations
      requiredN = numpy.zeros((numareas))
      for i in range(numareas):
            requiredN[i] = TotPop[y, i] - TotPop[y - 1, i] - Btot[y - 1, i] + Dtot[y - 1, i]
      
      # Preparing arrays for IPF
      NatN = numpy.zeros((2, lastage + 1))
      for s in range(2):
            for pc in range(lastage + 1):
               NatN[s, pc] = NatNpc[y - 1, s, pc]

      LocalPop0 = numpy.zeros((numareas, 2, lastage + 1))
      LocalDpc = numpy.zeros((numareas, 2, lastage + 1))
      for i in range(numareas):
            for s in range(2):
               for pc in range(lastage + 1):
                  LocalPop0[i, s, pc] = Pop0[y - 1, i, s, pc]
                  LocalDpc[i, s, pc] = Dpc[y - 1, i, s, pc]
      
      labelint = intervallabel[y]
      labely = y
      labelq = q

      # Generate IPF
      scaledIM = numpy.zeros((numareas, 2, lastage + 1))
      scaledOM = numpy.zeros((numareas, 2, lastage + 1))

      # Call the NetMigAdjustment2 function to generate IPF data (scaled Inward & Outward Migration)
      scaledIM, scaledOM = NetMigAdjustment2(labely, prelimIMpc, prelimOMpc, NatN, requiredN, LocalPop0, LocalDpc, scaledIM, scaledOM, numareas, lastage, maxziter, smallnumber, wb_wt_Log)

      # Allocate fitted migration to migration arrays (+++ IMpc, OMpc are written +++)
      for i in range(numareas):
         for s in range(2):
            for pc in range(lastage + 1):
               IMpc[y - 1, i, s, pc] = scaledIM[i, s, pc]
               OMpc[y - 1, i, s, pc] = scaledOM[i, s, pc]

      # END-OF-INTERVAL POPULATIONS
      for i in range(numareas):
         for s in range(2):
            for pc in range(lastage + 1):
               Pop1[y - 1, i, s, pc] = Pop0[y - 1, i, s, pc] - Dpc[y - 1, i, s, pc] + IMpc[y - 1, i, s, pc] - OMpc[y - 1, i, s, pc]

      # Difference between the end-of-interval population calculated above & Those of the previous iteration are compared
      totpop1_OK = 0
      pop1_OK = numpy.zeros((numareas, 2, lastage + 1))
      discrepancy = numpy.zeros((numareas, 2, lastage + 1))
      if (q > 1):
         for i in range(numareas):
            for s in range(2):
               for pc in range(lastage + 1):
                     discrepancy[i, s, pc] = abs(Pop1[y - 1, i, s, pc] - previousPop1[i, s, pc])
                     if (discrepancy[i, s, pc] < smallnumber):
                        pop1_OK[i, s, pc] = 0
                     else:
                        pop1_OK[i, s, pc] = 1
                     totpop1_OK += pop1_OK[i, s, pc]
      
      # Previous iteration loop value of the final population are updated
      for i in range(numareas):
         for s in range(2):
            for pc in range(lastage + 1):
               previousPop1[i, s, pc] = Pop1[y - 1, i, s, pc]
      
      # Convergence Check
      if q > 1:
         if (totpop1_OK == 0):
            break
   
   # Write out preliminary and scale net migration
   row = 2 + (((lastage + 2) * 2) * y) - ((lastage + 1) * 2)
   for s in range(2):
      for pc in range(lastage + 1):
         row += 1
         col = 3
         for i in range(numareas):
               wb_wt_CheckMig.cell(row, col).value = prelimIMpc[i, s, pc] - prelimOMpc[i, s, pc]
               wb_wt_CheckMig.cell(row, col + numareas + 3).value = IMpc[y - 1, i, s, pc] - OMpc[y - 1, i, s, pc]
         col += 1
         wb_wt_CheckMig.cell(row, col + numareas + 3).value = NatNpc[y - 1, s, pc]
   
   # Write out projected deaths
   row = 2 +  (((lastage + 2) * 2) * y) - ((lastage + 1) * 2)
   for s in range(2):
      for pc in range(lastage + 1):
         row += 1
         col = 3
         for i in range(numareas):
               wb_wt_CheckDeaths.cell(row, col).value = Dpc[y - 1, i, s, pc]
         col += 1
         wb_wt_CheckDeaths.cell(row, col).value = NatDpc[y - 1, s, pc]

   # Calculate total migration for population accounts output
   for i in range(numareas):
      totN[y - 1, i] = 0
      totINMIG[y - 1, i] = 0
      totOUTMIG[y - 1, i] = 0
      for s in range(2):
         for pc in range(lastage + 1):
            totN[y - 1, i] += (IMpc[y - 1, i, s, pc] - OMpc[y - 1, i, s, pc])
            totINMIG[y - 1, i] += IMpc[y - 1, i, s, pc]
            totOUTMIG[y - 1, i] += OMpc[y - 1, i, s, pc]
    
   # Adjustment of age subscripts (+++ Population is written +++)
   for i in range (numareas):
      for s in range(2):
         for pc in range(lastage + 1):
            Population[y, i, s, pc] = Pop1[y - 1, i, s, pc]
    
   # Total populations summed from age-sex populations (+++ totPopulation is written +++)
   for i in range(numareas):
      totPopulation[y, i] = 0
      for s in range(2):
         for a in range(numages):
            totPopulation[y, i] += Population[y, i, s, a]
    
   # Check for negative populations
   for i in range(numareas):
      for s in range(2):
         for a in range(numages):
            if Population[y, i, s, a] < 0:
               print("Error: Negative population detected in one or more age groups")

##### Write Projection Data

In [18]:
# Write the projection data into worksheet
wb_wt_AgeSexForecasts, wb_wt_Components = writeProj(wb_wt_AgeSexForecasts, wb_wt_Components, yearlabel, intervallabel, Areacode, Areaname, sexlabel, agelabel, \
                                                    numareas, numages, final, Population, totPopulation, Btot, Dtot, totN)
wb_wt.save(wt_loc)

## Projection Visualisation

In [20]:
# Visualisation of Summary of Population Projection with Selected Area and Projection Year
# Set male's data as left section and female's data as right section in the Pyramid Plot
Target_Area = "Queanbeyan"
Jump_Year = 2020
Proj_Year = 2035

# Write target data into sheet
wb_wt_Target = writeTarget(wb_wt_Target, Target_Area, Jump_Year, Proj_Year, Areaname, yearlabel, Population, numages, sexlabel, agelabel)
wb_wt.save(wt_loc)